In [2]:
import numpy as np
import scipy as sp
import sympy as sy
import matplotlib.pyplot as plt

from Main.funçoes import edo,Valor_esperado
from Main.estados import bases
from Main.operador import destruiçao,Identidade
from Main.class_tijolo import ObjQuantico

from IPython.display import display, Math

# Aula 10: 

Nesse primeioro material iremos seguir o tutorial do site :  https://docs.scipy.org/doc/scipy/tutorial/sparse.html direto do site da scipy.

## Introdução :

O módulo `scipy.sparse` e seus submódulos fornecem ferramentas para trabalhar com matrizes esparsas. 
Matrizes esparsas são um tipo especial de matriz em que apenas algumas posições contêm dados. 

Isso permite representações compactas, nas quais somente as posições com dados são registradas. Existem diversos formatos de matrizes esparsas, cada um fazendo um equilíbrio diferente entre compressão e funcionalidade. Para começar, vamos construir uma matriz esparsa simples no formato Coordenado (COO, ou `coo_array`) e compará-la com uma matriz densa:

In [ ]:
dense   = np.array([[1, 0, 0, 2], [0, 4, 1, 0], [0, 0, 5, 0]])
sparse  = sp.sparse.coo_array(dense)
print("Matriz Densa:")
print(sparse)

Observe que, em nossa matriz densa, temos cinco valores não nulos. Por exemplo, o número 2 está na posição 0,3, e o número 4 está na posição 1,1. Todos os outros valores são zeros. A matriz esparsa registra explicitamente esses cinco valores (veja os 5 elementos armazenados e o formato ((3,4)) e representa todos os zeros restantes como valores implícitos.


COO arrays cannot be subscripted or sliced:

In [ ]:
print(dense)
print(dense[2, 2])
# sparse[2, 2]

COO arrays cannot be subscripted or sliced:

But, other formats, such as the Compressed Sparse Row (CSR) csr_array support slicing and element indexing:

In [ ]:
sparse  = sp.sparse.csr_array(dense)
print(sparse)

## Convertendo a classe base

In [40]:
class ObjQuantico_esparso:
    def __init__(self, data,latex_representation=None):
        self.dados = data
        self.latex_representation = latex_representation

    def definir_dados(self, data):
        self.dados = data

    def full(self):
        return self.dados.todense()
    
    def full_sparsa(self):
        return self.dados
    
    def dim(self):
        return len(self.dados.todense())
    
    def expM(self):
        # Convertendo para csc matriz pq é mais eficiente
        matrix = sp.sparse.csc_matrix(self.dados)
        matrix = sp.sparse.linalg.expm(matrix)
        return ObjQuantico_esparso(matrix) 
    
    def dag(self):
        return ObjQuantico_esparso(self.dados.transpose().conjugate())
    
    def traço(self):
        return float(self.dados.trace().real)
    
    def Autovalores(self):
        return np.linalg.eigvals(self.dados.todense())
    
    def Autovetores(self):
        return np.linalg.eig(self.dados.todense())[1]
    
    def AutoValor_Vetor(self):
        return np.linalg.eig(self.dados.todense())
    
    def __repr__(self):
        if self.latex_representation:
            display(Math(self.latex_representation))
        else:
            display(Math(sy.latex(sy.Matrix(self.dados.todense()))))
        return f"ObjQuantico: dim ={self.dim()} , shape = {self.dados.todense().shape}" 
                
    def __add__(self, other):
        if isinstance(other, ObjQuantico_esparso):  
            # Soma os dados de dois objetos ObjQuantico_esparso
            return ObjQuantico_esparso(self.dados + other.dados)
        else:
            raise TypeError(f"Soma não suportada entre {type(other)} e ObjQuantico")
        
    def __sub__(self, other):
        if isinstance(other, ObjQuantico_esparso):  
            # Subtração entre duas instâncias de ObjQuantico
            return ObjQuantico_esparso(self.dados - other.dados)
        else:
            raise TypeError(f"Subtração não suportada entre {type(other)} e ObjQuantico")
    
    def __mul__(self, other):
        # Multiplicação para diferentes tipos
        if isinstance(other, ObjQuantico_esparso):  
            # Multiplicação matricial com outra instância de ObjQuantico
            return ObjQuantico_esparso(np.dot(self.dados, other.dados))
        elif np.isscalar(other):  # Multiplicação por escalar
            return ObjQuantico_esparso(self.dados * other)
        else:
            raise TypeError(f"Multiplicação não suportada entre {type(other)} e ObjQuantico")

    def __rmul__(self, other):
        if np.isscalar(other):  # Multiplicação reversa por escalar
            return ObjQuantico_esparso(self.dados * other)
        else:
            raise TypeError(f"Multiplicação não suportada entre {type(other)} e ObjQuantico")
    
    def __truediv__(self, other):
        if isinstance(other, (int, float)):  # Divisão por um número escalar
            return ObjQuantico_esparso(self.dados / other)
        else:
            raise TypeError(f"Divisão não suportada entre {type(other)} e ObjQuantico")
    
    def __rtruediv__(self, other):
        if isinstance(other, (int, float)):  # Divisão invertida por um número escalar
            return ObjQuantico_esparso(other / self.dados)
        else:
            raise TypeError(f"Divisão não suportada entre {type(other)} e ObjQuantico")     
         
    def __pow__(self, power):
        if not isinstance(power, int) or power < 0:
            raise ValueError("A potência deve ser um inteiro não negativo.")
        
        # Obter a matriz identidade no formato esparso
        size = self.dados.shape[0]  # Número de linhas
        result = sp.sparse.identity(size, format='csr')  # Identidade esparsa no formato CSR
        
        # Multiplicação iterativa
        for _ in range(power):
            result = result @ self.dados  # Multiplicação esparsa eficiente
        return ObjQuantico_esparso(result)
    
    def __matmul__(self, other):
        """Implementa o operador @ para o produto tensorial."""
        if isinstance(other, ObjQuantico_esparso):
            return ObjQuantico_esparso(sp.sparse.kron(self.full_sparsa(), other.full_sparsa()))
        else:
            raise TypeError(f"Operador @ não suportado entre {type(self)} e {type(other)}")

In [ ]:
dense   = np.array([[1j, 0,  0], [0, 4, 0j], [0, 0+1j, 1]])
dense   = 2*dense  - dense.T
dense   = sp.sparse.csr_array(dense)
print(dense)

###  Plote

In [ ]:
# __repr__()
spar_var = ObjQuantico_esparso(dense)
spar_var

###  Full_sparsa

````python
def full_sparsa(self):
    return self.dados
````
Metodo novo para rotrno a matris esparsa no formado sparso emquanto que o full() continua retornando a matriz esparça completa.

In [ ]:
# .full_sparsa()
print(spar_var.full_sparsa())

###  Dim

````python
def dim(self):
        return len(self.dados.todense())
````
Estamos calculado ca dimensão da matrix densa 

In [ ]:
# dim()
spar_var.dim()

###  Dag

````python
def dag(self):
    return ObjQuantico_esparso(np.conjugate(self.dados.T))
````
vamos mudar as funçoes do numpa para scipy sparso

In [ ]:
print(spar_var.full_sparsa().transpose())

In [ ]:
print(spar_var.full_sparsa().conjugate())

In [ ]:
def dag(self):
    return ObjQuantico_esparso(self.dados.transpose().conjugate())

print(spar_var.full_sparsa().transpose().conjugate())

###  Traço

````python
def traço(self):
    return np.trace(self.dados).real
````
vamos mudar as funçoes do numpy para scipy sparso

In [ ]:
def traço(self):
    return self.dados.trace().real

spar_var.full_sparsa().trace().real

###  AutoValor

````python
    def Autovalores(self):
        return np.linalg.eigvals(self.dados)
````
vamos mudar as funçoes do numpy para scipy sparso

In [ ]:
spar_var.full()

In [ ]:
sp.linalg.eig(spar_var.full())

Ha alguma limitaçoes no scipy.sparse.linalg.eigs , a principal é que ele não consegue calcular todos os autovalores e auto vetores, das matrizes esparsas. 

In [ ]:
# Nesse enxemplo para k >1 ja não fuciona pq a matriz 3x3 
sp.sparse.linalg.eigs(spar_var.full_sparsa(),k=1)

In [ ]:
def Autovalores(self):
    return np.linalg.eigvals(self.dados.todense())

np.linalg.eigvals(spar_var.full())

o mesmo vale apara as outra funções

In [ ]:
def Autovetores(self):
    return np.linalg.eig(self.dados.todense())[1]

def AutoValor_Vetor(self):
    return np.linalg.eig(self.dados.todense())

## Convertendo todos os operadores

### Identidade 

```python
def Identidade(N):
    matriz = np.identity(N)
    return ObjQuantico(matriz) 
```

In [ ]:
sp.sparse.identity(3).toarray()

Criado a função 

In [4]:
def Identidade(N,sparsa=False):
    if sparsa ==True:
        matriz = sp.sparse.identity(N)
        return ObjQuantico_esparso(matriz)
    else:
        matriz = np.identity(N)
        return ObjQuantico(matriz) 

In [ ]:
N = 3 
I = Identidade(N,sparsa=True)
I

In [ ]:
I+I #Soma

In [ ]:
I-I #Substração

In [ ]:
(1+7j)*I#Multiplicaçao

In [ ]:
I/10

In [ ]:
(1j*I)**2

### Destruição

```python
def destruiçao(N):
    subdiag = np.sqrt(np.arange(1, N))# Monta os elementos na subdiagonal
    dt      = np.diag(subdiag, k=1) # Operador de destruição
    return ObjQuantico(dt)
```


In [22]:
def destruiçao(N,sparsa=False):
    if sparsa ==True:
        subdiag = np.sqrt(np.arange(1, N))
        matriz  = sp.sparse.diags(subdiag,offsets=1)
        return ObjQuantico_esparso(matriz)
    else:
        subdiag = np.sqrt(np.arange(1, N)) # Monta os elementos na subdiagonal
        matriz  = np.diag(subdiag, k=1) # Operador de destruição
        return ObjQuantico(matriz) 

### Criação

In [17]:
def criaçao(N,sparsa=False):
    return  destruiçao(N,sparsa=sparsa).dag()    
   

### Operador x e P

In [14]:
def operador_p(N,sparsa=False):
    return -1j*(destruiçao(N,sparsa) - criaçao(N,sparsa))/np.sqrt(2)   

In [ ]:
print(operador_p(3,sparsa=True).full_sparsa())

In [23]:
def operador_x(N,sparsa=False):
    return (destruiçao(N,sparsa) + criaçao(N,sparsa))/np.sqrt(2) 

In [ ]:
print(operador_x(3,sparsa=True).full_sparsa())

### Matrizes de pauli

In [26]:
def pauliX(sparsa=False):
    latex_representation = r"$$ \hat{\sigma_x} $$"
    if sparsa ==  False:
        m = np.array([[ 0, 1 ],[ 1, 0 ]])
        return ObjQuantico(m,latex_representation)
    else:
        # Criando a matriz diretamente no formato esparso COO
        data = [1, 1]
        row = [0, 1]
        col = [1, 0]
        m   = sp.sparse.coo_array((data, (row, col)), shape=(2, 2))
        return ObjQuantico_esparso(m,latex_representation) 

def pauliY(sparsa=False):
    latex_representation = r"$$ \hat{\sigma_y} $$"
    
    if sparsa:
        # Criando a matriz diretamente no formato esparso COO
        data = [-1j, 1j]
        row = [0, 1]
        col = [1, 0]
        m   = sp.sparse.coo_array((data, (row, col)), shape=(2, 2))
        return ObjQuantico_esparso(m, latex_representation)
    else:
        # Criando a matriz no formato denso
        m = np.array([[0, -1j], [1j, 0]])
        return ObjQuantico(m, latex_representation)

def pauliZ(sparsa=False):
    latex_representation = r"$$ \hat{\sigma_z} $$"
    
    if sparsa:
        # Criando a matriz diretamente no formato esparso COO
        data = [1, -1]
        row = [0, 1]
        col = [0, 1]
        m   =  sp.sparse.coo_array((data, (row, col)), shape=(2, 2))
        return ObjQuantico_esparso(m, latex_representation)
    else:
        # Criando a matriz no formato denso
        m = np.array([[1, 0], [0, -1]])
        return ObjQuantico(m, latex_representation)

In [ ]:
m = pauliY(sparsa=True)
m

In [ ]:
print(m.full_sparsa())

In [ ]:
m.traço()

In [ ]:
m.dag()

In [ ]:
m @ m.dag()

### Matriz densidade

In [94]:
def matrizdensidade(probabilities=None, estados=None, puro=True,sparsa=False): 
    if puro == True:
        rho = estados*estados.dag()
        if sparsa==True:
            return ObjQuantico_esparso(sp.sparse.coo_array(rho.full()))
        else:
            return rho
    
    else:    
        # Verificar se as probabilidades somam 1
        if not np.isclose(sum(probabilities), 1):
            raise ValueError("As probabilidades devem somar 1.")
        
        # Verificar se cada estado está normalizado
        for state in estados:
            if not np.isclose(np.linalg.norm(state.full()), 1):
                raise ValueError("Todos os estados devem ser normalizados.")
             
        # Criar a matriz densidade
        dimensao = estados[0].full().shape[0] # pega o primeiro estado, em seguida a dimensao do estado
        rho = np.zeros((dimensao, dimensao), dtype=complex)
        for p, state in zip(probabilities, estados):
            rho += p * np.outer(state.full(), state.dag().full())  # |ψ⟩⟨ψ|
            
        if sparsa == True:
            return ObjQuantico_esparso(sp.sparse.coo_array(rho))
        else:
            return ObjQuantico(rho)

In [ ]:
estado = bases(N=3,n=1)
estado

In [ ]:
matrizdensidade(estados=estado) # densa

In [ ]:
matrizdensidade(estados=estado,sparsa=True) # Esparsa

In [ ]:
matrizdensidade(estados=estado,sparsa=True).full_sparsa() # Esparsa

In [ ]:
estados = [bases(N=3,n=0),bases(N=3,n=1),bases(N=3,n=2)]
probabilidades = [0.25,0.25,0.5]

matrizdensidade(probabilidades, estados, puro=False,sparsa=False)

In [ ]:
print(matrizdensidade(probabilidades, estados, puro=False,sparsa=True).full_sparsa())

## Convertendo todos os estado

### Bases

In [21]:
def bases(N,n,sparsa=False):
    if sparsa == False:
        estadoinicial = np.zeros(shape=(N, 1),dtype=complex)
        estadoinicial[n, 0] = 1
        return ObjQuantico(estadoinicial) 
    else:
        estadoinicial = sp.sparse.lil_array((N, 1),dtype=complex)
        estadoinicial[n:n+1] =  1 
        return ObjQuantico_esparso(estadoinicial) 


In [ ]:
N = 5
n = 1
bases(N,n)

In [ ]:
bases(N,n,sparsa=True)

### kets e bras

In [64]:
def ket(entrada,sparsa=False):
    if isinstance(entrada, (int, float)) or (isinstance(entrada, str) and entrada in ('0', '1')):
        
        latex_representation = rf"$$ \ket{entrada} $$"
        
        if sparsa == False:
            dados = bases(N=2,n=int(entrada),sparsa=sparsa).full()
            return ObjQuantico(dados, latex_representation)
        else:
            dados = bases(N=2,n=int(entrada),sparsa=sparsa).full_sparsa()
            return ObjQuantico_esparso(dados, latex_representation)
    else:
        print("Entrada invalida / tente usar outra função( sugestão bases) ")
    

In [ ]:
ket(0),ket(1)

In [ ]:
ket(0,sparsa=True).full_sparsa(),ket(1,sparsa=True).full_sparsa()

In [74]:
def bra(entrada,sparsa=False):
    if isinstance(entrada, (int, float)) or (isinstance(entrada, str) and entrada in ('0', '1')):
        
        latex_representation = rf"$$ \bra{entrada} $$"
        
        if sparsa == False:
            dados = bases(N=2,n=int(entrada),sparsa=sparsa).dag().full()
            return ObjQuantico(dados, latex_representation)
        else:
            dados = bases(N=2,n=int(entrada),sparsa=sparsa).dag().full_sparsa()
            return ObjQuantico_esparso(dados, latex_representation)
    else:
        print("Entrada invalida / tente usar outra função( sugestão bases) ")
    

In [ ]:
bra(0),bra(1),bra(0).full(),bra(1).full()

In [ ]:
bra(0,sparsa=True),bra(1,sparsa=True)

### Fock

In [79]:
def Fock(N, n=0,sparsa=False):
    return bases(N, n,sparsa)

In [ ]:
Fock(N=10)

### Coerente

In [64]:
def coerente(N,alpha,metodo ="operador",sparsa = False):
    
    if metodo == "operador" :
        estado  = bases(N,0,sparsa) # estado inicinal no vacuo
        D       = alpha * destruiçao(N,sparsa).dag() - np.conj(alpha) * destruiçao(N,sparsa)
        D       = D.expM()
        return D*estado
        
    elif metodo == "analitico":    # implementar o metodo de matrizes esparsas
        if sparsa ==False:
            estado  = np.zeros(shape=(N,1),dtype=complex)
            n       = np.arange(N)
            estado[:,0] = np.exp(-(abs(alpha) ** 2 )/ 2.0) * (alpha**n)/np.sqrt(sp.special.factorial(n))
            return ObjQuantico(estado)     
        else:
            estado = sp.sparse.lil_array((N, 1),dtype=complex)
            estado[:] = np.exp(-(abs(alpha) ** 2 )/ 2.0) * (alpha**np.arange(N))/np.sqrt(sp.special.factorial(np.arange(N)))
            return ObjQuantico_esparso(estado)   
    else:
        raise TypeError(
            "A opção de método tem as seguintes opções :'operador' ou 'analitico'")

In [72]:
alpha = 0.1
N=3
coerente(N,alpha,metodo="analitico")

<IPython.core.display.Math object>

ObjQuantico: dim =3 , shape = (3, 1)

In [73]:
coerente(N,alpha,metodo="analitico",sparsa=True)

<IPython.core.display.Math object>

ObjQuantico: dim =3 , shape = (3, 1)

In [74]:
coerente(N,alpha,metodo="operador",sparsa=True)

<IPython.core.display.Math object>

ObjQuantico: dim =3 , shape = (3, 1)

# Convertendo o solve

### TODO List: Criar um Solver para Resolver EDO Linear de Primeira Ordem

1. **Definir a EDO Linear de Primeira Ordem:**
    - Definir a equação diferencial dx/dt = A(t)x + b, onde A é uma matriz esparsa e b é um vetor.

2. **Importar Bibliotecas Necessárias:**
    - Importar `numpy` para operações numéricas.
    - Importar `scipy.sparse` para manipulação de matrizes esparsas.
    - Importar `scipy.integrate` para métodos de integração.

3. **Definir a Função para a EDO:**
    - Criar uma função que representa a EDO dx/dt = A(t)x + b.

4. **Configurar Parâmetros Iniciais:**
    - Definir os valores iniciais para t0, tf, e y0.
    - Definir a matriz esparsa A e o vetor b.

5. **Escolher o Método de Integração:**
    - Selecionar um método de integração adequado, como `solve_ivp` do `scipy.integrate`.

6. **Implementar o Solver:**
    - Implementar o solver utilizando `solve_ivp` para resolver a EDO.

7. **Testar o Solver:**
    - Testar o solver com diferentes matrizes esparsas A e vetores b.
    - Verificar a precisão e eficiência do solver.

8. **Visualizar os Resultados:**
    - Plotar os resultados da solução da EDO.
    - Analisar o comportamento da solução ao longo do tempo.

9. **Documentar o Código:**
    - Adicionar comentários e documentação ao código para facilitar a compreensão.

10. **Otimizar o Código:**
     - Otimizar o código para melhorar a performance, se necessário.

11. **Salvar e Compartilhar:**
     - Salvar o notebook e compartilhar com colegas para revisão e feedback.